In [1]:
import chromadb
from chromadb.api.models.Collection import Collection
from llama_cpp import Llama
import qdrant_client
from qdrant_client import QdrantClient
from typing import List, Optional, Union
from langchain.vectorstores import Qdrant, Chroma
from langchain.docstore.document import Document
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.document_loaders import (
    CSVLoader,
    EverNoteLoader,
    PDFMinerLoader,
    TextLoader,
    UnstructuredEPubLoader,
    UnstructuredHTMLLoader,
    UnstructuredMarkdownLoader,
    UnstructuredODTLoader,
    UnstructuredPowerPointLoader,
    UnstructuredWordDocumentLoader,
)

In [2]:
collection = "all-documents"
model = "models/saiga2_7b_gguf/model-q2_K.gguf"
EMBEDDER_NAME = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
llama_model = Llama(
    model_path=model,
    n_ctx=2000,
    n_parts=1,
)
embeddings = HuggingFaceEmbeddings(model_name=EMBEDDER_NAME)

llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from models/saiga2_7b_gguf/model-q2_K.gguf (version GGUF V2)
llama_model_loader: - tensor    0:                token_embd.weight q2_K     [  4096, 32002,     1,     1 ]
llama_model_loader: - tensor    1:              blk.0.attn_q.weight q2_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    2:              blk.0.attn_k.weight q2_K     [  4096,  1024,     1,     1 ]
llama_model_loader: - tensor    3:              blk.0.attn_v.weight q3_K     [  4096,  1024,     1,     1 ]
llama_model_loader: - tensor    4:         blk.0.attn_output.weight q3_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_gate.weight q3_K     [  4096, 14336,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.ffn_up.weight q3_K     [  4096, 14336,     1,     1 ]
llama_model_loader: - tensor    7:            blk.0.ffn_down.weight q3_K     [ 14336,  4096,     1,     1 

In [3]:
LOADER_MAPPING: dict = {
    ".csv": (CSVLoader, {}),
    ".doc": (UnstructuredWordDocumentLoader, {}),
    ".docx": (UnstructuredWordDocumentLoader, {}),
    ".enex": (EverNoteLoader, {}),
    ".epub": (UnstructuredEPubLoader, {}),
    ".html": (UnstructuredHTMLLoader, {}),
    ".md": (UnstructuredMarkdownLoader, {}),
    ".odt": (UnstructuredODTLoader, {}),
    ".pdf": (PDFMinerLoader, {}),
    ".ppt": (UnstructuredPowerPointLoader, {}),
    ".pptx": (UnstructuredPowerPointLoader, {}),
    ".txt": (TextLoader, {"encoding": "utf8"}),
}

def load_single_document(file_path: str):
    """
    Загружаем один документ.
    :param file_path:
    :return:
    """
    ext: str = "." + file_path.rsplit(".", 1)[-1]
    assert ext in LOADER_MAPPING
    loader_class, loader_args = LOADER_MAPPING[ext]
    loader = loader_class(file_path, **loader_args)
    return loader.load()[0]

In [24]:
query = "Какие требования к актуализации ВНД"
path = "/home/timur/Загрузки/AI VDN/ПН_РСК_001_02_Положение_о_системе_ВНД.docx"
documents: List[Document] = [load_single_document(path)]

text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

ids: List[str] = [
    f"{doc.metadata['source'].split('/')[-1].replace('.txt', '')}{i}"
    for i, doc in enumerate(docs)
]

documents = [doc.page_content for doc in docs]
metadatas = [doc.metadata for doc in docs]

Created a chunk of size 1117, which is longer than the specified 500
Created a chunk of size 1975, which is longer than the specified 500
Created a chunk of size 1494, which is longer than the specified 500
Created a chunk of size 2952, which is longer than the specified 500
Created a chunk of size 1492, which is longer than the specified 500
Created a chunk of size 555, which is longer than the specified 500
Created a chunk of size 1099, which is longer than the specified 500
Created a chunk of size 2596, which is longer than the specified 500


In [ ]:
# LANGCHAIN QDRANT

In [5]:
client = qdrant_client.QdrantClient(path="./qdrant_langchain_test")
qdrant = Qdrant(
    client=client, 
    collection_name=collection, 
    embeddings=embeddings
)

In [7]:
qdrant = qdrant.from_documents(
    docs,
    embeddings,
    path="./qdrant_langchain_test",
    collection_name=collection,
)
qdrant

In [8]:
found_docs = qdrant.similarity_search(query)

print(found_docs[0].metadata)
print(found_docs[0].page_content)

{'source': 'load_texts/ПН_РСК_001_02_Положение_о_системе_ВНД.docx'}
использование стандартизованной терминологии (корпоративного глоссария);

процедуры и действия, предусмотренные документом, должны быть реально выполнимыми.

Для оптимизации сроков основного согласования ВНД целесообразно привлекать на этапе разработки документа работников подразделений – обязательных участников основного согласования, а также работников ООР для методологической поддержки.

Результатом этапа разработки ВНД является:

проект ВНД;

проект распорядительного документа о введении в действие ВНД;

пояснительная записка или краткая презентация о целесообразности разработки ВНД и его ключевых положениях (при необходимости).

По результатам разработки все проекты ВНД должны пройти процедуру предварительного согласования, включающую рассмотрение:

непосредственным руководителем Разработчика на соответствие содержания ВНД целям его разработки;


In [ ]:
# LANGCHAIN CHROMA

In [5]:
client = chromadb.PersistentClient(path="./chroma_langchain_test")
db = Chroma(
    client=client,
    collection_name=collection,
    embedding_function=embeddings,
)

In [6]:
db.get()

{'ids': [], 'embeddings': None, 'metadatas': [], 'documents': []}

In [7]:
db = db.from_documents(
    docs, 
    embeddings,
    persist_directory="./chroma_langchain_test",
    collection_name=collection,
)

In [8]:
docs = db.similarity_search(query)

# print results
print(docs[0].metadata)
print(docs[0].page_content)

{'source': 'load_texts/ПН_РСК_001_02_Положение_о_системе_ВНД.docx'}
Необходимость в разработке / актуализации ВНД по приведенным выше основаниям выявляется владельцами и участниками процессов, органами управления Общества, уполномоченными государственными и иными организациями / лицами. 

Выявленная потребность в регламентации процессов может при необходимости фиксироваться в плане-графике разработки / актуализации ВНД по направлению деятельности. 

Требования к разработке проекта ВНД

Процесс разработки ВНД в Обществе может быть инициирован:


In [ ]:
# LANGCHAIN CLICKHOUSE

In [20]:
from langchain.vectorstores import Clickhouse, ClickhouseSettings

In [22]:
settings = ClickhouseSettings(
    host="localhost",
    port=8123,
    username="default",
    password="",
    table="vector_table"
)

In [23]:
db = Clickhouse.from_documents(
    docs, 
    embeddings, 
    config=settings
)

DatabaseError: :HTTPDriver for http://localhost:8123 returned response code 500)
 Code: 80. DB::Exception: Annoy index second argument must be String. (INCORRECT_QUERY) (version 23.4.2.11 (official build))


In [ ]:
docs = db.similarity_search(query)

# print results
print(docs[0].metadata)
print(docs[0].page_content)

In [12]:
# CHROMA

In [9]:
client: chromadb.PersistentClient = chromadb.PersistentClient(path="./chroma_test")
db: Collection = client.get_or_create_collection(collection)
db

Collection(name=all-documents)

In [10]:
db.add(
    documents=documents,
    metadatas=metadatas,
    ids=ids
)

In [11]:
docs = db.query(
    query_texts=[query],
    n_results=4
)

In [12]:
data: dict = {}
for doc, text in zip(docs["metadatas"][0], docs["documents"][0]):
    document: str = f'Документ - {doc["source"].split("/")[-1]}'
    if document in data:
        data[document] += "\n" + text
    else:
        data[document] = text
list_data: list = [f"{doc}\n\n{text}" for doc, text in data.items()]
print("\n\n\n".join(list_data))

Документ - ПН_РСК_001_02_Положение_о_системе_ВНД.docx

- документы 4-го уровня, определяющие последовательность выполнения действий внутри одного функционального блока/подразделения.

В Обществе предусмотрены следующие виды ВНД:

Таблица №1
ВНД утверждается и вводится в действие органом управления / должностным лицом в соответствии со следующими правилами:
Для удобства управления все ВНД, создаваемые в Обществе, классифицируются по видам, типам доступа и сроку действия.

В Обществе, в зависимости от назначения, разрабатываются ВНД следующих видов:
По результатам разработки все проекты ВНД должны пройти процедуру предварительного согласования, включающую рассмотрение:

непосредственным руководителем Разработчика на соответствие содержания ВНД целям его разработки;


In [ ]:
# QDRANT

In [7]:
client = QdrantClient(path="./qdrant_test", embedding_models=embeddings)

In [8]:
from qdrant_client.http.models import Distance, VectorParams

client.create_collection(
    collection_name=collection,
    vectors_config=VectorParams(size=4, distance=Distance.DOT),
)

True

In [9]:
client.get_collection(collection_name=collection)

CollectionInfo(status=<CollectionStatus.GREEN: 'green'>, optimizer_status=<OptimizersStatusOneOf.OK: 'ok'>, vectors_count=0, indexed_vectors_count=0, points_count=0, segments_count=1, config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=4, distance=<Distance.DOT: 'Dot'>, hnsw_config=None, quantization_config=None, on_disk=None), shard_number=None, sharding_method=None, replication_factor=None, write_consistency_factor=None, read_fan_out_factor=None, on_disk_payload=None, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=None, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=1), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), quantization_config=None), payload_schema={})

In [12]:
client.add(
    collection_name=collection,
    documents=documents,
    metadata=metadatas,
    ids=ids
)

AssertionError: Collection have incompatible vector params: size=4 distance=<Distance.DOT: 'Dot'> hnsw_config=None quantization_config=None on_disk=None

In [10]:
search_result = client.query(
    collection_name=collection,
    query_text=query
)
print(search_result)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 77.7M/77.7M [01:35<00:00, 817kiB/s]


ValueError: Dense vector fast-bge-small-en is not found in the collection

In [11]:
client.search(
    collection_name=collection,
    query_vector=(query, [0.2, 0.1, 0.9, 0.7]),
    limit=3,
)

ValueError: Dense vector Какие требования к актуализации ВНД is not found in the collection

In [7]:
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.storage.storage_context import StorageContext
from llama_index.embeddings import HuggingFaceEmbedding
from llama_index import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    ServiceContext,
)

In [8]:
embeddings_index = HuggingFaceEmbedding(model_name=EMBEDDER_NAME)
service_context = ServiceContext.from_defaults(
    llm=llama_model,
    embed_model=embeddings_index,
)

AttributeError: 'Llama' object has no attribute 'system_prompt'

In [ ]:
client = chromadb.PersistentClient(path="./chroma_llama_index_test")

In [ ]:
db = ChromaVectorStore(client=client, collection_name=collection)
storage_context = StorageContext.from_defaults(vector_store=db)
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, service_context=service_context
)
index

In [ ]:
query_engine = index.as_query_engine(
    streaming=True
)
response = query_engine.query(query)
response.print_response_stream()